# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

**Reminder about our [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)**

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt', encoding = 'utf8') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt', encoding = 'utf8') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    precision = len(set.intersection(gold, pred)) / len(pred)
    
    recall = len(set.intersection(gold, pred)) / len(gold)
    
    F1 = 2 * precision * recall / (precision + recall)
    
    print(f"Precision: {round(precision*100,2)}%, Recall: {round(recall*100,2)}%, F1 score: {round(F1*100,2)}%")

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

Precision: 60.0%, Recall: 100.0%, F1 score: 75.0%


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for rows in df.itertuples():
        yield (rows.sentence_id, rows.beg, rows.end)

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [8]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


In [9]:
print(list(spans_dev_gold)[0])
print(list(spans_dev_gold)[-1])

('0956-019', 0, 1)
('0999-003', 31, 32)


In [10]:
df_dev.head()

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


In [11]:
print(list(gold_spans(df_dev))[0])
print(list(gold_spans(df_dev))[-1])

('0946-000', 2, 3)
('1161-010', 1, 3)


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [12]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

def spans(df):
    for row_id in df.sentence_id.unique():
        sent_doc = nlp(df.sentence[df.sentence_id == row_id].values[0])
        for entity in sent_doc.ents:
            yield (row_id, entity.start, entity.end)
        
evaluation_report(spans_dev_gold, set(spans(df_dev)))


Precision: 52.14%, Recall: 70.22%, F1 score: 59.84%


In [13]:
set(spans(df_dev))

{('0991-007', 29, 30),
 ('0948-015', 0, 3),
 ('0980-002', 0, 1),
 ('0968-002', 20, 23),
 ('1044-007', 11, 12),
 ('1069-003', 7, 8),
 ('1112-002', 16, 17),
 ('1112-008', 16, 17),
 ('0957-019', 7, 9),
 ('0979-012', 4, 5),
 ('0963-023', 8, 11),
 ('0966-070', 0, 1),
 ('0958-007', 2, 4),
 ('1083-002', 2, 3),
 ('1082-002', 13, 14),
 ('1157-002', 19, 20),
 ('1075-002', 13, 14),
 ('0966-164', 22, 23),
 ('0966-021', 0, 1),
 ('1060-021', 3, 4),
 ('1072-014', 18, 19),
 ('1159-003', 19, 20),
 ('1060-004', 0, 1),
 ('1097-007', 3, 6),
 ('1096-015', 3, 4),
 ('0958-010', 4, 6),
 ('0954-002', 29, 31),
 ('1058-030', 3, 4),
 ('1103-015', 68, 69),
 ('1078-005', 7, 8),
 ('1009-018', 31, 32),
 ('1022-001', 0, 1),
 ('1135-029', 6, 7),
 ('0980-006', 33, 34),
 ('0989-002', 26, 28),
 ('0957-006', 15, 16),
 ('0978-003', 6, 8),
 ('1018-003', 18, 19),
 ('0966-119', 0, 1),
 ('1064-007', 34, 35),
 ('0960-003', 5, 6),
 ('1033-002', 22, 23),
 ('1148-005', 13, 14),
 ('1006-007', 0, 1),
 ('1126-004', 1, 4),
 ('1025-011'

## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [14]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [15]:
# TODO: Write code here to do your analysis
error_report(df_train[:500], 
             set(gold_spans(df_train[:500])),
             set(spans(df_train[:500])))

Sentence: BRUSSELS 1996-08-22
  FP: 1996-08-22
Sentence: The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
  FP: The European Commission
  FP: Thursday
  FN: European Commission
Sentence: Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
  FP: Wednesday
  FP: the European Union 's
  FN: European Union
Sentence: He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .
  FP: the European Union
  FN: European Union
Sentence: He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary mov

In [16]:
evaluation_report(set(gold_spans(df_train[:500])),
                  set(spans(df_train[:500])))

Precision: 65.22%, Recall: 81.0%, F1 score: 72.26%


*Most errors were observed with DATES & NUMBERS. Preprocessing these out gave us an improvement of about 10% in the precision. The next biggest error label we removed was MONEY and PERCENT, which gave us a further 12% improvement in precision. Finally, we saw a lot of errors due to nationality, which we tried to correct by preprocessing NORP & LOC, but it didnt yield any improvement in the precision and also didn't seem to remove the nationality related errors. So, we skipped that preprocessing step. In the end to achieve a minor improvement and get $\sim$15% improvement in F1 score vs problem 2 we also preprocessed QUANTITY.*

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. To filter out specific labels it is useful to know the named entity label scheme, which can be found in the [model's documentation](https://spacy.io/models/en#en_core_web_sm). You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [17]:
# TODO: Write code here to improve the span prediction from Problem 2
def spans_improved(df):
    for row_id in df.sentence_id.unique():
        sent_doc = nlp(df.sentence[df.sentence_id == row_id].values[0])
        for entity in sent_doc.ents:
            if entity.label_ not in ['DATE', 'CARDINAL', 'MONEY', 'PERCENT', 'QUANTITY']:
                yield (row_id, entity.start, entity.end)
            
error_report(df_train[:500], 
             set(gold_spans(df_train[:500])),
             set(spans_improved(df_train[:500])))

Sentence: The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
  FP: The European Commission
  FN: European Commission
Sentence: Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
  FP: the European Union 's
  FN: European Union
Sentence: He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .
  FP: the European Union
  FN: European Union
Sentence: He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .
  FP: EU Farm
  FN: EU
Sentence: Fischler proposed

In [18]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [19]:
spacy.explain('ORDINAL')

'"first", "second", etc.'

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

In [20]:
evaluation_report(set(gold_spans(df_train[:500])),
                  set(spans_improved(df_train[:500])))

Precision: 87.47%, Recall: 81.0%, F1 score: 84.11%


In [21]:
evaluation_report(spans_dev_gold, set(spans_improved(df_dev)))

Precision: 79.79%, Recall: 70.14%, F1 score: 74.65%


Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [22]:
# TODO: Write code here to store the predicted spans in a new data frame

NER_dev = set(spans_improved(df_dev))

df_dev_NER = pd.DataFrame(NER_dev, columns = ['sentence_id', 'beg', 'end'])
df_dev_NER.sort_values(by=['sentence_id','beg']).reset_index(drop=True)
df_dev_sent = df_dev[['sentence_id','sentence']].drop_duplicates()
df_dev_NER = pd.merge(df_dev_sent, df_dev_NER, on = 'sentence_id', how = 'inner')
df_dev_NER['label'] = '--NME--'

In [23]:
df_dev_NER

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,--NME--
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,--NME--
2,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,--NME--
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,--NME--
...,...,...,...,...,...
5196,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
5197,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
5198,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
5199,1161-009,The DSE all share price index closed 2.73 poin...,1,2,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [24]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for rows in df.itertuples():
        yield (rows.sentence_id, rows.beg, rows.end, rows.label)

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

**Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.**

In [25]:
# TODO: Write code here to implement the baseline

df_dev_NBL = df_dev_NER.copy()
labels = []
for rows in df_dev_NBL.itertuples():
    labels.append("_".join(rows.sentence.split()[rows.beg:rows.end]))
df_dev_NBL.label = labels

In [26]:
df_dev_NBL.head()

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,LONDON
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indian
2,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire


In [27]:
evaluation_report(set(gold_mentions(df_dev)),
                  set(gold_mentions(df_dev_NBL)))

Precision: 29.57%, Recall: 25.99%, F1 score: 27.67%


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [28]:
with bz2.open('kb.tsv.bz2', 'rt', encoding = 'utf8') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [29]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [30]:
# TODO: Write code here to implement the "most probable entity" method.

def MPE(df, entity):
    mentions = df.loc[df.mention == entity]
    if len(mentions) > 0:
        return mentions.entity.values[0]
    else:
        return '--NME--'
    
df_dev_KB = df_dev_NER.copy()
#entity = ["_".join(rows.sentence.split()[rows.beg:rows.end]) for rows in df_dev_KB.itertuples()]
entity = [" ".join(rows.sentence.split()[rows.beg:rows.end]) for rows in df_dev_KB.itertuples()]
KB_entity = [MPE(df_kb, e) for e in entity]
df_dev_KB.label = KB_entity

evaluation_report(set(gold_mentions(df_dev)),
                  set(gold_mentions(df_dev_KB)))

Precision: 61.18%, Recall: 53.78%, F1 score: 57.24%


In [31]:
df_dev_KB.head()

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,London
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
2,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset_County_Cricket_Club
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [32]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [33]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [34]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [35]:
# TODO: Write code here to implement the context-sensitive disambiguation method

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


mentions = df_contexts.mention.unique()

clf_dict = {}

for i in mentions:
    prior = df_kb.loc[df_kb.mention == i].sort_values('entity').prob.values
    
    pipe = Pipeline([
        ('vec', CountVectorizer()),
        ('clf', MultinomialNB(class_prior=prior)),
    ])
    
    clf_dict[i] = pipe.fit(df_contexts.context[df_contexts.mention == i],
                           df_contexts.entity[df_contexts.mention == i])


In [36]:
clf_dict

{'1970': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf', MultinomialNB(class_prior=array([0.5, 0.5])))]),
 '1990 World Cup': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf',
                  MultinomialNB(class_prior=array([0.97916667, 0.02083333])))]),
 '1992': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf',
                  MultinomialNB(class_prior=array([0.98571429, 0.01428571])))]),
 '1996': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf',
                  MultinomialNB(class_prior=array([0.94100295, 0.00294985, 0.0560472 ])))]),
 '2000 Summer Olympics': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf',
                  MultinomialNB(class_prior=array([9.99696878e-01, 3.03122158e-04])))]),
 '95': Pipeline(steps=[('vec', CountVectorizer()),
                 ('clf', MultinomialNB(class_prior=array([0.5, 0.5])))]),
 'ABC': Pipeline(steps=[('vec', CountVectorizer()),
              

In [37]:
df_dev_context = df_dev_NER.copy()

labels = []

for rows in df_dev_context.itertuples():
    context =' '.join(rows.sentence.split()[rows.beg:rows.end])
    
    if context in clf_dict:
        MNB_clf = clf_dict[context]
        labels.append(MNB_clf.predict([rows.sentence])[0])
    elif context in df_kb.mention.values:
        labels.append(df_kb.loc[df_kb.mention == context, 'entity'].values[0])
    else:
        labels.append('--NME--')
        
df_dev_context.label = labels

In [38]:
df_dev_context

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,London
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,Caribbean
2,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset_County_Cricket_Club
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club
...,...,...,...,...,...
5196,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",10,12,--NME--
5197,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",7,9,--NME--
5198,1161-007,"Brokers said blue chips like IDLC , Bangladesh...",13,15,--NME--
5199,1161-009,The DSE all share price index closed 2.73 poin...,1,2,Dhaka_Stock_Exchange


In [39]:
evaluation_report(set(gold_mentions(df_dev)),
                  set(gold_mentions(df_dev_context)))

Precision: 62.62%, Recall: 55.04%, F1 score: 58.59%


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 5.1:** In Problem&nbsp;3, you did an error analysis on the task of recognizing text spans mentioning named entities. Summarize your results. Pick one type of error that you observed. How could you improve the model&rsquo;s performance on this type of error? What resources (such as domain knowledge, data, compute) would you need to implement this improvement?

*We summarized this a bit during the analysis but we recall here that the biggest source of FPs and FNs seemed to be DATES & NUMBERS. We preprocessed them out, and saw a significant jump in the model's precision. The preprocessing approach was made by observing the FPs & FNs and figure out a logical pattern. So essentially, we needed enough data to recognise a pattern. Domain knowledge wasnt the most important factor here because we are not bringing in context for this exercise.*

**RQ 5.2:** Thinking back about Problem&nbsp;6, explain what the word *context* refers to in the task addressed there, and how context can help to disambiguate between different entities. Suggest other types of context that you could use for disambiguation.

*In this task context was a combination of 5 tokens to the left and right of the mention. This approach can help to classify the entity relationship by not just using the mention as the insput but also the surrounding text. In this exercise we were able to create a dictionary that assigns probabilities to different entities connected to a mention based on context surrounding said mention. To suggest an improvement, why only use the 5 surrounding words, we can also use word clusters to define context. For eg, if the document I am looking to classify has historical references to American history, then Lincoln would be linked to Abraham Lincoln. On the other hand, if it were a geographical text then maybe Lincoln, NB would be a better classification.*

**RQ 5.3:** One type of entity mentions that we did not cover explicitly in this lab are pronouns. As an example, consider the sentence pair *Ruth Bader Ginsburg was an American jurist*. *She served as an associate justice of the Supreme Court from 1993 until her death in 2020*. What facts would you want to extract from this sentence pair? How do pronouns make fact extraction hard?

*Facts from sentence pair:* 
- *Ruth Bader Ginsburg was the full name of 1 person*
- *Ruth Bader Ginsburg was female*
- *Ruth Bader Ginsburg's profession was a jurist*
- *Ruth Bader Ginsburg was an American* 
- *From 1993-2020 Ruth Bader Ginsburg was serving at the Supreme Court*
- *During her time at the Supreme Court she was an associate justice*
- *Ruth Bader Ginsburg died in 2020*

*Pronouns make it hard because they are used generically to describe any person according to their gender/identity. This creates a lot of ambiguity in linking named entities to information linked to their pronoun in the document. It is very challenging to create a separate contextual gold standard that assigns facts to the same entity. In this example, Ruth Bader Ginsburg is the mention and her name, first name, last name, pronoun, nick-name etc can all be clustered together as different entities.*


**This was the last lab in the Text Mining course. Congratulations! 🥳**